In [2]:
from data import Mappings, Databases
from collections import OrderedDict
import yaml
import pymysql
import pandas as pd

def compare(s1, s2):
    return len(s1 - s2), len(s1 & s2), len(s2 - s1)

umls_db = pymysql.connect(db='UMLS2014AB_CoMap', host='127.0.0.1', user='root', password='root', port=3306)

In [6]:
project = 'safeguard'
with open("../projects/{}/events.yaml".format(project)) as f:
    events = yaml.load(f)
with open("../projects/{}/config.yaml".format(project)) as f:
    databases = databases = Databases.of_config(yaml.load(f))
with open("../projects/{}/mappings.yaml".format(project)) as f:
    mappings = Mappings.of_raw_data_and_normalize(yaml.load(f), events, databases)

# {DB: {code}}
mapping_codes = {
    db: mappings.all_codes(db)
    for db, _ in databases
}

In [50]:
sabs = ["ICD10", "ICD10CM", "ICD10PCS"]
query = """
    select distinct code from MRCONSO where sab = %s
"""
icd10_codes = {}
with umls_db.cursor() as cursor:
    for sab in sabs:
        cursor.execute(query, [sab])
        icd10_codes[sab] = {r[0] for r in cursor.fetchall()}

In [68]:
df = pd.Series(OrderedDict([
    ('ICD10', len(mapping_codes['GePaRD'] & voc_codes['ICD10'])),
    ('ICD10CM', len(mapping_codes['GePaRD'] & voc_codes['ICD10CM'])),
    ('ICD10+ICD10CM', len(mapping_codes['GePaRD'] & (voc_codes['ICD10'] | voc_codes['ICD10CM']))),
    ('mapping', len(mapping_codes['GePaRD'])),
])).to_frame('N')
df['%'] = (df.N / df.N.mapping).map('{:.1%}'.format)
df

,N,%
ICD10,66,91.7%
ICD10CM,69,95.8%
ICD10+ICD10CM,72,100.0%
mapping,72,100.0%


In [4]:
sabs = ["ICPC", "ICPC2P", "ICPC2EENG", "ICPC2EDUT", "ICPC2ICD10ENG"]
query = """
    select distinct code from MRCONSO where sab = %s
"""
icpc_codes = {}
with umls_db.cursor() as cursor:
    for sab in sabs:
        cursor.execute(query, [sab])
        icpc_codes[sab] = {r[0] for r in cursor.fetchall()}

In [9]:
df = pd.Series(OrderedDict([
    ('ICPC', len(mapping_codes['IPCI'] & icpc_codes['ICPC'])),
    ('ICPC2P', len(mapping_codes['IPCI'] & icpc_codes['ICPC2P'])),
    ('ICPC2EENG', len(mapping_codes['IPCI'] & (icpc_codes['ICPC2EENG']))),
    ('ICPC2EDUT', len(mapping_codes['IPCI'] & (icpc_codes['ICPC2EDUT']))),
    ('ICPC2ICD10ENG', len(mapping_codes['IPCI'] & (icpc_codes['ICPC2ICD10ENG']))),
    ('mapping', len(mapping_codes['IPCI'])),
])).to_frame('N')
df['%'] = (df.N / df.N.mapping).map('{:.1%}'.format)
df

,N,%
ICPC,7,100.0%
ICPC2P,0,0.0%
ICPC2EENG,7,100.0%
ICPC2EDUT,0,0.0%
ICPC2ICD10ENG,0,0.0%
mapping,7,100.0%
